### This notebook shows how to get the altitude from a list of longtitudes and latitudes. And how to calculate the distance between each pair.

### Calculating the distance between pairwise coordinates

In [1]:
import pandas as pd
import numpy as np
import sklearn.neighbors
import requests

In [2]:
# Create two dataframes with city names and lat-long in degrees
df_cities_a = pd.DataFrame({
    'city_A' :     ['Atlanta', 'Aspen', 'Albuquerque', 'Ann Arbor'],
    'latitude_A':  [ 33.75,     39.19,   35.08,         42.28],
    'longitude_A': [ -84.39,    -106.82, -106.65,       -83.74]
})
df_cities_b = pd.DataFrame({
    'city_B':      ['Boston', 'Baltimore', 'Berkley', 'Bellevue'],
    'latitude_B' : [ 42.36,    39.29,       37.87,     47.61],
    'longitude_B': [ -71.06,   -76.61,      -122.27,   -122.20]
})

#### add columns with radians for latitude and longitude


In [3]:
df_cities_a[['lat_radians_A','long_radians_A']] = (
    np.radians(df_cities_a.loc[:,['latitude_A','longitude_A']])
)
df_cities_b[['lat_radians_B','long_radians_B']] = (
    np.radians(df_cities_b.loc[:,['latitude_B','longitude_B']])
)

In [4]:
dist = sklearn.neighbors.DistanceMetric.get_metric('haversine')
dist_matrix = (dist.pairwise
    (df_cities_a[['lat_radians_A','long_radians_A']],
     df_cities_b[['lat_radians_B','long_radians_B']])*3959
)
# Note that 3959 is the radius of the earth in miles
df_dist_matrix = (
    pd.DataFrame(dist_matrix,index=df_cities_a['city_A'], 
                 columns=df_cities_b['city_B'])
)

/home/imspecies/.local/lib/python3.8/site-packages/sklearn/neighbors/_distance_metric.py:10: FutureWarning: sklearn.neighbors.DistanceMetric has been moved to sklearn.metrics.DistanceMetric in 1.0. This import path will be removed in 1.3
  warnings.warn(


In [5]:
df_dist_matrix

city_B,Boston,Baltimore,Berkley,Bellevue
city_A,,,,
Atlanta,936.076298,576.797573,2126.980759,2172.370998
Aspen,1870.061460,1609.131867,839.063817,963.873663
Albuquerque,1967.911506,1671.139388,887.886902,1178.937434
Ann Arbor,647.250084,426.212109,2042.197926,1896.343851


### Get the altitude from latitude and longtitude

This function keeps sending request until it can get the altitude (Therefore, might be a little bit time consuming)

In [19]:
def get_altitudes(lat:[float], lon:[float]) -> [float]:
    """Request altitudes from open-elevation.com

    Args:
        lat (List[float]): Latitudes
        long (List[float]): Longitudes

    Returns:
        List[float]: Altitudes
    """
    assert len(lat) == len(lon)
    post_query = {
        "locations": [dict(latitude=lat, longitude=lon) for lat, lon in zip(lat, lon)]
    }
    headers = {"accept": "application/json", "content-type": "application/json"}
    while True:
        try:
            r = requests.post(
                "https://api.open-elevation.com/api/v1/lookup",
                headers=headers,
                json=post_query,
            )
            r = r.json()
            break
        except Exception:
            print(f"Failed query. Status code: {r.status_code}. Retrying...")
    altitudes = [x["elevation"] for x in r["results"]]
    return altitudes

In [22]:
df_cities_a["altitude_A"] = get_altitudes(df_cities_a["latitude_A"], df_cities_a["longitude_A"])
df_cities_b["altitude_B"] = get_altitudes(df_cities_b["latitude_B"], df_cities_b["longitude_B"])

Failed query. Status code: 504. Retrying...
Failed query. Status code: 504. Retrying...
Failed query. Status code: 504. Retrying...
Failed query. Status code: 504. Retrying...
Failed query. Status code: 504. Retrying...
Failed query. Status code: 504. Retrying...
Failed query. Status code: 504. Retrying...
Failed query. Status code: 504. Retrying...
Failed query. Status code: 504. Retrying...


In [23]:
df_cities_a

,city_A,latitude_A,longitude_A,lat_radians_A,long_radians_A,altitude_A
0,Atlanta,33.75,-84.39,0.589049,-1.472883,327
1,Aspen,39.19,-106.82,0.683995,-1.864361,2417
2,Albuquerque,35.08,-106.65,0.612262,-1.861394,1509
3,Ann Arbor,42.28,-83.74,0.737925,-1.461539,269


In [24]:
df_cities_b

,city_B,latitude_B,longitude_B,lat_radians_B,long_radians_B,altitude_B
0,Boston,42.36,-71.06,0.739321,-1.240231,27
1,Baltimore,39.29,-76.61,0.685740,-1.337097,12
2,Berkley,37.87,-122.27,0.660956,-2.134014,63
3,Bellevue,47.61,-122.20,0.830951,-2.132792,27
